In [1]:
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
import torch
from src.data.argoverse_datamodule import ArgoverseDataModule
from src.data.argoverse_dataset import ArgoverseDataset
from src.data.dummy_datamodule import DummyDataModule
from src.models.WIMP import WIMP
from torch.utils.data import DataLoader


In [2]:
args = {"IFC":True, "add_centerline":False, "attention_heads":4, "batch_norm":False, "batch_size":1, "check_val_every_n_epoch":3, 
          "dataroot":'./data/nuscene_data/', "dataset":'argoverse', "distributed_backend":'ddp', "dropout":0.0, 
          "early_stop_threshold":5, "experiment_name":'example', "gpus":1, "gradient_clipping":True, "graph_iter":1, 
          "hidden_dim":512, "hidden_key_generator":True, "hidden_transform":False, "input_dim":2, "k_value_threshold":10, 
          "k_values":[6, 5, 4, 3, 2, 1], "lr":0.0001, "map_features":False, "max_epochs":200, "mode":'train', "model_name":'WIMP', 
          "no_heuristic":False, "non_linearity":'relu', "num_layers":4, "num_mixtures":6, "num_nodes":1, "output_conv":True, "output_dim":2, 
          "output_prediction":True, "precision":32, "predict_delta":False, "resume_from_checkpoint":None, 
          "scheduler_step_size":[60, 90, 120, 150, 180], "seed":None, "segment_CL":False, "segment_CL_Encoder":False, 
          "segment_CL_Encoder_Gaussian":False, "segment_CL_Encoder_Gaussian_Prob":False, "segment_CL_Encoder_Prob":True, 
          "segment_CL_Gaussian_Prob":False, "segment_CL_Prob":False, "use_centerline_features":True, "use_oracle":False, "waypoint_step":5, 
          "weight_decay":0.0, "workers":8, "wta":False, "draw_image" : False, "remove_high_related_score" : True, "maximum_delete_num" : 3, 
       "is_LRP" : False}

In [12]:
train_loader = ArgoverseDataset(
    args["dataroot"],
    mode="train",
    delta=args["predict_delta"],
    map_features_flag=args["map_features"],
    social_features_flag=True,
    heuristic=(not args["no_heuristic"]),
    ifc=args["IFC"],
    is_oracle=args["use_oracle"],
)
train_loader[6]

./data/nuscene_data/
train
num_social_agents:  20


({'agent_features': array([[  0.5343375 ,   0.14167508],
         [  0.5343375 ,   0.14167508],
         [  0.5343375 ,   0.14167508],
         [  0.5343375 ,   0.14167508],
         [  0.5343375 ,   0.14167508],
         [  0.5343375 ,   0.14167508],
         [  0.5343375 ,   0.14167508],
         [  0.5343375 ,   0.14167508],
         [  0.5343375 ,   0.14167508],
         [ 12.560943  , -53.32498   ],
         [ 11.388047  , -47.96742   ],
         [ 10.2141695 , -42.610046  ],
         [  9.082373  , -37.59393   ],
         [  7.9495935 , -32.578003  ],
         [  6.864975  , -27.903166  ],
         [  5.7803564 , -23.228327  ],
         [  4.7316217 , -18.554642  ],
         [  3.681714  , -13.880168  ],
         [  2.6329796 ,  -9.206484  ],
         [  1.5832635 ,  -4.532991  ],
         [  0.5343375 ,   0.14167508]], dtype=float32),
  'ifc_helpers': {'agent_oracle_centerline': array([[  5.5597234, -41.316418 ],
          [  7.3943844, -49.915455 ],
          [  9.164228 , -58.

In [3]:
train_loader = ArgoverseDataset(
    args["dataroot"],
    mode="train",
    delta=args["predict_delta"],
    map_features_flag=args["map_features"],
    social_features_flag=True,
    heuristic=(not args["no_heuristic"]),
    ifc=args["IFC"],
    is_oracle=args["use_oracle"],
)

val_loader = ArgoverseDataset(
    args["dataroot"],
    mode="val",
    delta=args["predict_delta"],
    map_features_flag=args["map_features"],
    social_features_flag=True,
    heuristic=(not args["no_heuristic"]),
    ifc=args["IFC"],
    is_oracle=args["use_oracle"],
)

test_loader = ArgoverseDataset(
    args["dataroot"],
    mode="test",
    delta=args["predict_delta"],
    map_features_flag=args["map_features"],
    social_features_flag=True,
    heuristic=(not args["no_heuristic"]),
    ifc=args["IFC"],
    is_oracle=args["use_oracle"],
)

train_dataset = DataLoader(
    train_loader,
    batch_size=args["batch_size"],
    num_workers=args["workers"],
    pin_memory=True,
    collate_fn=ArgoverseDataset.collate,
    shuffle=True,
    drop_last=True,
)

val_dataset = DataLoader(
    val_loader,
    batch_size=args["batch_size"],
    num_workers=args["workers"],
    pin_memory=True,
    collate_fn=ArgoverseDataset.collate,
    shuffle=False,
    drop_last=False,
)

trest_dataset = DataLoader(
    test_loader,
    batch_size=args["batch_size"],
    num_workers=args["workers"],
    pin_memory=True,
    collate_fn=ArgoverseDataset.collate,
    shuffle=False,
    drop_last=False,
)
# dm = ArgoverseDataset(args)

./data/nuscene_data/
train
./data/nuscene_data/
val
./data/nuscene_data/
test


In [4]:
for d in val_dataset:
    print(d)
    break

num_social_agents: num_social_agents: num_social_agents: num_social_agents: num_social_agents: num_social_agents:       000000


num_social_agents: 


num_social_agents:   020

num_social_agents: num_social_agents: num_social_agents: num_social_agents: num_social_agents:  num_social_agents:    0 0 00
3

11
num_social_agents: 

 num_social_agents:  00

num_social_agents:  0


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/workspace/MotionPrediction/WIMP/src/data/argoverse_dataset.py", line 295, in __getitem__
    social_max_pad = np.max(social["social_oracle_centerline_lengths"])
  File "<__array_function__ internals>", line 5, in amax
  File "/opt/conda/lib/python3.8/site-packages/numpy/core/fromnumeric.py", line 2705, in amax
    return _wrapreduction(a, np.maximum, 'max', axis, None, out,
  File "/opt/conda/lib/python3.8

In [ ]:
model = WIMP(args)
model.load_state_dict(torch.load("experiments/example/checkpoints/epoch=122.ckpt")['state_dict'])
model = model.cuda()

In [ ]:
def get_metric(metric_dict, ade,fde,mr,loss):
    metric_dict["ade"] += ade
    metric_dict["fde"] += fde
    metric_dict["mr"] += mr
    metric_dict["loss"] += loss
    metric_dict["length"]+=1

# a = torch.FloatTensor([1])
# get_metric(origina_model_metric, a,a,a,a)
# origina_model_metric

In [ ]:
#  optimizer는 선언하지 않았습니다.
#  따라서 아마 model weight에 gradient는 계속 쌓이겠지만 저희가 중요한 것은 adjacency matrix의 graident이므로 상관 없을 것으로 예측됩니다.
import os
import math
import copy
import torch.nn as nn
Relu = nn.ReLU()

save_foler = "ResultsImg/"

save_XAI = save_foler + "/XAI/"
save_attention = save_foler + "/attention"

slicing = lambda a, idx: torch.cat((a[:, :idx], a[:, idx+1:]), axis=1)



In [ ]:

abs_min = lambda weight : torch.argmin(abs(weight)[1:]).item()
abs_max = lambda weight : torch.argmax(abs(weight)[1:]).item()
simple_min = lambda weight : torch.argmin(weight[1:]).item()
simple_max = lambda weight : torch.argmax(weight[1:]).item()


names = ["abs_min", "abs_max", "simple_min","simple_max"]
for name_idx, function in enumerate([abs_min, abs_max, simple_min, simple_max]):
    origina_model_metric  ={
            "ade" : torch.FloatTensor([0]).cuda(),                                                                                                                                         "mr" : torch.FloatTensor([0]).cuda(),
            "loss" : torch.FloatTensor([0]).cuda(),
        "length" : 0
    }

    DA_model_metric = [
        {
            "ade" : torch.FloatTensor([0]).cuda(),
            "fde" : torch.FloatTensor([0]).cuda(),
            "mr" : torch.FloatTensor([0]).cuda(),
            "loss" : torch.FloatTensor([0]).cuda(),
            "length" : 0    
        },
        {
            "ade" : torch.FloatTensor([0]).cuda(),
            "fde" : torch.FloatTensor([0]).cuda(),
            "mr" : torch.FloatTensor([0]).cuda(),
            "loss" : torch.FloatTensor([0]).cuda(),
            "length" : 0    
        },
        {
            "ade" : torch.FloatTensor([0]).cuda(),
            "fde" : torch.FloatTensor([0]).cuda(),
            "mr" : torch.FloatTensor([0]).cuda(),
            "loss" : torch.FloatTensor([0]).cuda(),
            "length" : 0
        },
    ] # 하나씩 지우면서 metric을 잴것임



    for batch_idx, batch in enumerate(tqdm(val_dataset)):
        input_dict, target_dict = batch[0], batch[1]

        # get cuda
        input_dict['agent_features'] = input_dict['agent_features'].cuda()
        input_dict['social_features'] = input_dict['social_features'].cuda()
        input_dict['social_label_features'] = input_dict['social_label_features'].cuda()
        input_dict['adjacency'] = input_dict['adjacency'].cuda()
        input_dict['label_adjacency'] = input_dict['label_adjacency'].cuda()
        input_dict['num_agent_mask'] = input_dict['num_agent_mask'].cuda()
        input_dict['ifc_helpers']['agent_oracle_centerline'] = input_dict['ifc_helpers']['agent_oracle_centerline'].cuda()
        input_dict['ifc_helpers']['agent_oracle_centerline_lengths'] = input_dict['ifc_helpers']['agent_oracle_centerline_lengths'].cuda()
        input_dict['ifc_helpers']['social_oracle_centerline'] = input_dict['ifc_helpers']['social_oracle_centerline'].cuda()
        input_dict['ifc_helpers']['social_oracle_centerline_lengths'] = input_dict['ifc_helpers']['social_oracle_centerline_lengths'].cuda()
        input_dict['ifc_helpers']['agent_oracle_centerline'] = input_dict['ifc_helpers']['agent_oracle_centerline'].cuda()
        target_dict['agent_labels'] = target_dict['agent_labels'].cuda()


        preds, waypoint_preds, all_dist_params, attention, adjacency = model(**input_dict)
        loss, (ade, fde, mr) = model.eval_preds(preds, target_dict, waypoint_preds)
        get_metric(origina_model_metric, ade,fde,mr,loss)


        input_dict['adjacency'].requires_grad = True
        input_dict['adjacency'].retain_grad()

        adjacency.retain_grad()
        print(torch.sum(model.decoder.value_generator.weight))
        loss.backward()
        print(torch.sum(model.decoder.value_generator.weight))
        batch_preds = preds



        for idx in range(args["batch_size"]):
            if args["draw_image"]:
                weight = adjacency.grad[idx][0].cpu().numpy()
                att = attention[idx].cpu().numpy()
                agent_features = input_dict['agent_features'][idx].cpu().numpy()
                social_features = input_dict['social_features'][idx].cpu().numpy()
                # target = target_dict['agent_labels'][idx].cpu().numpy()
                preds = batch_preds[idx][:,:,:,:2][0].cpu().detach().numpy()
                city_name = input_dict['ifc_helpers']['city'][idx]
                rotation = input_dict['ifc_helpers']['rotation'][idx].numpy()
                translation = input_dict['ifc_helpers']['translation'][idx].numpy()
                XAI_utils.draw_attention(agent_features, social_features, preds, city_name, rotation, translation, 
                                               weight = copy.deepcopy(att), draw_future=True, save_fig = True, 
                                                 = save_attention + "/" + str(batch_idx) + "_" + str(idx) + ".png")

                XAI_utils.draw(agent_features, social_features, preds, city_name, rotation, translation, 
                                   weight = copy.deepcopy(weight), draw_future=True, save_fig = True, 
                                   save_name = save_XAI + "/" + str(batch_idx) + "_" + str(idx) + ".png")

            if args["remove_high_related_score"]:
                weight = adjacency.grad[idx][0]
                for i in range(args["maximum_delete_num"]):
    #                 print(batch_idx, input_dict["social_features"].shape)
                    if len(input_dict["social_features"][0] > 1):
                        arg = function(weight)
    #                     arg_max = torch.argmin(weight[1:]).item()

                        weight = torch.cat((weight[:arg+1], weight[arg+2:]))                                                                                                                                                                                                                                                                                                                                                                             
                        input_dict["social_features"]                                        = slicing(input_dict["social_features"], arg)
                        input_dict["num_agent_mask"]                                    = slicing(input_dict["num_agent_mask"], arg+1)
                        input_dict["ifc_helpers"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        ['social_oracle_centerline']             = slicing(input_dict["ifc_helpers"]['social_oracle_centerline'], arg)
                        input_dict["ifc_helpers"]['social_oracle_centerline_lengths']  = slicing(input_dict["ifc_helpers"]['social_oracle_centerline_lengths'], arg)
                    else:
                        break

                    with torch.no_grad():
                        preds, waypoint_preds, all_dist_params, att_weights, adjacency = model(input_dict["agent_features"],
                                                    input_dict["social_features"],
                                                    None,
                                                    input_dict["num_agent_mask"],
                                                    ifc_helpers ={
                                                        "social_oracle_centerline": input_dict["ifc_helpers"]['social_oracle_centerline'], 
                                                        "social_oracle_centerline_lengths": input_dict["ifc_helpers"]['social_oracle_centerline_lengths'],
                                                        "agent_oracle_centerline": input_dict["ifc_helpers"]["agent_oracle_centerline"],
                                                        "agent_oracle_centerline_lengths": input_dict["ifc_helpers"]["agent_oracle_centerline_lengths"]
                                                    })
                        loss, (ade, fde, mr) = model.eval_preds(preds, target_dict, waypoint_preds)
                        get_metric(DA_model_metric[i], ade,fde,mr,loss)

In [ ]:
model = WIMP(args)
model.load_state_dict(torch.load("experiments/example/checkpoints/epoch=122.ckpt")['state_dict'])
model.cuda()
# model.eval()


In [ ]:
# model.eval()
preds, waypoint_preds, all_dist_params, att_weights, adjacency = model(**input_dict)
loss, (ade, fde, mr) = model.eval_preds(preds, target_dict, waypoint_preds)
loss

In [ ]:
model.encoder.xy_conv_filters[0].weight.grad
